In [1]:
# To install crewAI, you need to have Python >=3.10 and <=3.13 installed on your system:
import sys
print(sys.version)

3.11.13 (main, Jun  4 2025, 08:57:29) [GCC 11.4.0]


In [ ]:
!pip install crewai

In [4]:
from crewai import Agent, Task, Crew, LLM

# Warning control
import warnings
warnings.filterwarnings('ignore')

import os
from google.colab import userdata

os.environ["GROQ_API_KEY"] = userdata.get('GROQ_API_KEY')
os.environ["GEMINI_API_KEY"] = userdata.get('GEMINI_API_KEY')

In [5]:
groq_llm = LLM(model="groq/llama3-70b-8192", temperature=0)

#deepseek_llm = LLM(model="groq/deepseek-r1-distill-llama-70b", temperature=0)
gemini_llm = LLM(
    model="gemini/gemini-2.0-flash",
    temperature=0
)

###Creating Agents

Create instance of the Agent class with the desired properties, `role`, `goal` and `backstory`

- `role`: agent's function within the crew
- `goal`: agent's objective. Guides agent's decision making process
- `backstory`: provides context to agent's role and goal. Helps in interation and collaboration.

### Define the Tasks

Tasks are **specific assignments**, completed by agents.
They **provide all necessary details for execution**, such as a `description`, `expected_output` , `agent` responsible, tools, etc.

- `description`: Tells about what task entails
- `expected_output`: Tells about what task output should look like
- `agent`: Agent responsible for the task

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

## Palnner

In [6]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic",
    llm= gemini_llm,
    allow_delegation=False,
    verbose=True
)

In [7]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

## Writer

In [8]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    llm= gemini_llm,
    allow_delegation=False,
    verbose=True
)

In [9]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
        "3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer
)

## Editor

In [10]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    llm= gemini_llm,
    allow_delegation=False,
    verbose=True
)

In [11]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

### Step 05. Forming the crew

Crew is **group of AI agents**, collaborating to perform set of tasks.


- `tasks`: list of tasks
- `agents`: list of agents
- `process (optional)`: work flow (e.g., sequential, hierarchical) default : sequential
- `verbose=2` allows you to see all the logs of the execution.


    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.



In [12]:
crew = Crew(
    tasks=[plan, write, edit],
    agents=[planner, writer, editor],
    verbose=False
)



result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Content Planner                                                                                         │
│                                                                                                                 │
│  Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.            │
│  2. Identify the target audience, considering their interests and pain points.                                  │
│  3. Develop a detailed content outline including an introduction, key points, and a call to action.             │
│  4. Include SEO keywords and relevant data or sources.                                                          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Content Planner                                                                                         │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  ## Content Plan: Blog Article on Artificial Intelligence                                                       │
│                                                                                                                 │
│  **1. Target Audience:**                                                                                        │
│                                                                                                                 │
│  *   **Primary Audience:** Tech enthusiasts, business professionals, students, and individuals curious about    │
│  AI's impact on various industries.                                                                             │
│      *   **Interests:** Understanding AI applications, career opportunities in AI, the ethical implications of  │
│  AI, and the future of AI.                                                                                      │
│      *   **Pain Points:** Overwhelmed by technical jargon, difficulty understanding AI's practical              │
│  applications, concerns about job displacement, and confusion about the different types of AI.                  │
│  *   **Secondary Audience:** Investors, policymakers, and researchers interested in the latest AI advancements  │
│  and their potential impact on society and the economy.                                                         │
│      *   **Interests:** Investment opportunities in AI, regulatory frameworks for AI, and the societal impact   │
│  of AI.                                                                                                         │
│      *   **Pain Points:** Lack of clear understanding of AI's potential risks and benefits, difficulty in       │
│  evaluating AI investments, and concerns about the ethical implications of AI.                                  │
│                                                                                                                 │
│  **2. SEO Keywords:**                                                                                           │
│                                                                                                                 │
│  *   **Primary Keywords:** Artificial Intelligence, AI, Machine Learning, Deep Learning                         │
│  *   **Secondary Keywords:** AI applications, AI trends, AI ethics, AI future, AI in business, AI in            │
│  healthcare, AI in education, AI jobs, AI companies, AI development, AI algorithms, Generative AI, Large        │
│  Language Models, AI safety, Responsible AI.                                                                    │
│  *   **Long-Tail Keywords:** "What are the latest trends in artificial intelligence?", "How is AI used in       │
│  business?", "What are the ethical concerns of AI?", "How to learn artificial intelligence?", "Best AI          │
│  companies to work for", "Future of AI and its impact on society", "AI applications in healthcare industry",    │
│  "Generative AI tools for content creation", "How to build a career in AI", "Responsible AI development         │
│  practices".                                                                                                    │
│                                                                                                                 │
│  **3. Latest Trends, Key Players, and Noteworthy News:*

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Content Writer                                                                                          │
│                                                                                                                 │
│  Task: 1. Use the content plan to craft a compelling blog post on Artificial Intelligence.                      │
│  2. Incorporate SEO keywords naturally.                                                                         │
│  3. Sections/Subtitles are properly named in an engaging manner.                                                │
│  4. Ensure the post is structured with an engaging introduction, insightful body, and a summarizing             │
│  conclusion.                                                                                                    │
│  5. Proofread for grammatical errors and alignment with the brand's voice.                                      │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Content Writer                                                                                          │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  ```markdown                                                                                                    │
│  # The Ultimate Guide to Artificial Intelligence: Trends, Applications, and Future Impact                       │
│                                                                                                                 │
│  Artificial Intelligence (AI) is no longer a futuristic fantasy; it's a present-day reality rapidly             │
│  transforming our world. From self-driving cars to personalized medicine, AI is already deeply embedded in our  │
│  lives, and its influence is only set to grow. AI is projected to contribute a staggering $15.7 trillion to     │
│  the global economy by 2030, highlighting its immense potential. This article aims to provide a comprehensive   │
│  overview of AI, exploring its fundamentals, latest trends, diverse applications, and the profound impact it    │
│  will have on our future.                                                                                       │
│                                                                                                                 │
│  We'll delve into the different types of AI, from the narrow AI that powers our spam filters to the             │
│  hypothetical super AI that surpasses human intelligence. Whether you're a tech enthusiast, a business          │
│  professional, or simply curious about the AI revolution, this guide will equip you with the knowledge you      │
│  need to understand and navigate this exciting new era.                                                         │
│                                                                                                                 │
│  ## Section 1: Understanding the Fundamentals of AI                                                             │
│                                                                                                                 │
│  So, what exactly is Artificial Intelligence? At its core, **Artificial Intelligence** refers to the ability    │
│  of a computer or machine to mimic human cognitive functions such as learning, problem-solving, and             │
│  decision-making. It involves creating systems that can reason, understand language, recognize patterns, and    │
│  even learn from experience.                                                                                    │
│                                                                                                                 │
│  The history of **AI** is marked by periods of excitement and setbacks. The field emerged in the 1950s with     │
│  early successes in areas like game playing and symbolic reasoning. However, progress slowed down due to        │
│  limitations in computing power and a lack of data. In recent years, advancements in hardware, the              │
│  availability of massive datasets, and breakthroughs in algorithms have led to a resurgence of AI, driving      │
│  innovation across various industries.                                                                          │
│                                                                                                                 │
│  AI can be broadly categorized into three types:                                                                │
│                                                        

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Editor                                                                                                  │
│                                                                                                                 │
│  Task: Proofread the given blog post for grammatical errors and alignment with the brand's voice.               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Editor                                                                                                  │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  ```markdown                                                                                                    │
│  # The Ultimate Guide to Artificial Intelligence: Trends, Applications, and Future Impact                       │
│                                                                                                                 │
│  Artificial Intelligence (AI) is rapidly evolving from a futuristic concept into a present-day reality,         │
│  profoundly transforming our world. From revolutionizing industries to enhancing daily life, AI's influence is  │
│  undeniable and poised for continued growth. Experts project AI to contribute a staggering $15.7 trillion to    │
│  the global economy by 2030, underscoring its immense potential. This article offers a comprehensive overview   │
│  of AI, exploring its core principles, current trends, diverse applications, and the significant impact it      │
│  will have on our future.                                                                                       │
│                                                                                                                 │
│  We will explore the various types of AI, ranging from the narrow AI powering everyday tools like spam filters  │
│  to the theoretical concept of super AI surpassing human intelligence. Whether you're a tech enthusiast, a      │
│  business professional, or simply curious about the AI revolution, this guide aims to provide you with the      │
│  knowledge necessary to understand and navigate this exciting technological era.                                │
│                                                                                                                 │
│  ## Section 1: Understanding the Fundamentals of AI                                                             │
│                                                                                                                 │
│  So, what exactly is Artificial Intelligence? At its core, **Artificial Intelligence** refers to the            │
│  capability of a computer or machine to mimic human cognitive functions, including learning, problem-solving,   │
│  and decision-making. It involves creating systems designed to reason, comprehend language, recognize           │
│  patterns, and learn from practical experiences.                                                                │
│                                                                                                                 │
│  The history of **AI** is characterized by alternating periods of enthusiasm and challenges. The field          │
│  initially emerged in the 1950s, marked by early achievements in areas such as game playing and symbolic        │
│  reasoning. However, progress faced slowdowns due to limitations in computing power and data availability.      │
│  Recent advancements in hardware, the emergence of massive datasets, and breakthroughs in algorithmic           │
│  development have propelled a resurgence of AI, fostering innovation across diverse industries.                 │
│                                                                                                                 │
│  AI can be broadly classified into three distinct categories:                                                   │
│                                                        

In [13]:
from IPython.display import Markdown

# Extract markdown content from between the code blocks
markdown_content = result.raw.split('```markdown\n')[1].split('\n```')[0]
Markdown(markdown_content)

# The Ultimate Guide to Artificial Intelligence: Trends, Applications, and Future Impact

Artificial Intelligence (AI) is rapidly evolving from a futuristic concept into a present-day reality, profoundly transforming our world. From revolutionizing industries to enhancing daily life, AI's influence is undeniable and poised for continued growth. Experts project AI to contribute a staggering $15.7 trillion to the global economy by 2030, underscoring its immense potential. This article offers a comprehensive overview of AI, exploring its core principles, current trends, diverse applications, and the significant impact it will have on our future.

We will explore the various types of AI, ranging from the narrow AI powering everyday tools like spam filters to the theoretical concept of super AI surpassing human intelligence. Whether you're a tech enthusiast, a business professional, or simply curious about the AI revolution, this guide aims to provide you with the knowledge necessary to understand and navigate this exciting technological era.

## Section 1: Understanding the Fundamentals of AI

So, what exactly is Artificial Intelligence? At its core, **Artificial Intelligence** refers to the capability of a computer or machine to mimic human cognitive functions, including learning, problem-solving, and decision-making. It involves creating systems designed to reason, comprehend language, recognize patterns, and learn from practical experiences.

The history of **AI** is characterized by alternating periods of enthusiasm and challenges. The field initially emerged in the 1950s, marked by early achievements in areas such as game playing and symbolic reasoning. However, progress faced slowdowns due to limitations in computing power and data availability. Recent advancements in hardware, the emergence of massive datasets, and breakthroughs in algorithmic development have propelled a resurgence of AI, fostering innovation across diverse industries.

AI can be broadly classified into three distinct categories:

*   **Narrow or Weak AI:** This type of AI is engineered to execute specific tasks, such as filtering spam, providing product recommendations, or recognizing voice commands. It excels within its defined parameters but lacks broader, general intelligence.
*   **General or Strong AI:** This refers to AI possessing human-level intelligence, capable of performing any intellectual task that a human being can. While largely theoretical, it remains a significant pursuit for many AI researchers.
*   **Super AI:** This is a hypothetical form of AI that surpasses human intelligence in all facets, including creativity, problem-solving, and comprehensive understanding. It remains a topic of considerable debate and speculation.

Several key concepts are fundamental to the field of AI:

*   **Machine Learning (ML):** A subset of AI that empowers systems to learn from data without explicit programming. Machine learning encompasses several types:
    *   **Supervised Learning:** Training a model using labeled data to enable predictive capabilities.
    *   **Unsupervised Learning:** Identifying patterns and structures within unlabeled data.
    *   **Reinforcement Learning:** Training an agent to make decisions within an environment to maximize potential rewards.
*   **Deep Learning (DL):** A specialized type of machine learning employing artificial neural networks with multiple layers to analyze data. Deep learning has achieved notable success in areas such as image recognition and natural language processing.
*   **Natural Language Processing (NLP):** This field concentrates on enabling computers to understand, interpret, and generate human language. NLP is utilized in applications such as chatbots, machine translation, and sentiment analysis.
*   **Computer Vision:** This area of AI focuses on enabling computers to "see" and interpret images and videos. Computer vision is applied in areas such as object detection, facial recognition, and medical image analysis.

## Section 2: Latest Trends in Artificial Intelligence

The field of **Artificial Intelligence** is in constant flux, with new trends emerging and evolving rapidly. Keeping abreast of these developments is essential for understanding the current landscape and future trajectory of AI.

One of the most prominent trends is **Generative AI**. Generative AI models, including GPT-4, Bard, DALL-E 2, and Stable Diffusion, are capable of producing new content such as text, images, code, and music. These models are transforming industries by facilitating content creation, automating code generation, and enabling creative expression. For example, businesses are leveraging **Generative AI tools for content creation** to develop marketing materials, website content, and even complete articles.

**AI-powered Automation** represents another significant trend. AI is increasingly used to automate tasks across various industries, enhancing efficiency and reducing operational costs. Robotic Process Automation (RPA), for example, employs AI to automate repetitive tasks within business processes. AI-powered chatbots are also becoming more prevalent, providing instant customer support and managing routine inquiries.

**Edge AI** is also gaining momentum. Edge AI involves deploying AI models on edge devices, such as smartphones, IoT devices, and autonomous vehicles. This approach allows for quicker processing, reduced latency, and enhanced privacy, as data is processed locally rather than in the cloud.

As AI becomes more widespread, **AI Ethics and Governance** are receiving increased attention. Growing concerns exist regarding bias, fairness, transparency, and accountability within AI systems. Regulatory initiatives, such as the EU AI Act and the US AI Bill of Rights, aim to establish ethical guidelines and ensure responsible AI development and deployment. It's important to consider the **ethical concerns of AI** as the technology becomes more integrated into society.

**AI-driven Cybersecurity** is also on the rise. AI is being utilized to detect and prevent cyber threats, including malware, phishing attacks, and network intrusions. AI algorithms can analyze large volumes of data to identify anomalies and patterns indicative of malicious activity.

Finally, **Explainable AI (XAI)** is becoming increasingly vital. XAI seeks to make AI models more transparent and understandable, enabling users to comprehend how AI systems arrive at their decisions. This is particularly crucial in high-stakes applications, such as healthcare and finance, where trust and accountability are paramount.

## Section 3: Real-World Applications of AI

**AI applications** are revolutionizing industries and creating new opportunities across various sectors. Let's explore some key examples:

In **Business**, AI is enhancing customer service through chatbots and personalized recommendations. It's also transforming marketing with targeted advertising and lead generation. In finance, AI is used for fraud detection and algorithmic trading, improving efficiency and accuracy.

**AI in Healthcare** is demonstrating immense potential. AI-powered diagnostics can analyze medical images to detect diseases earlier and more accurately. AI is also accelerating drug discovery by identifying potential drug candidates and predicting their effectiveness. Personalized medicine uses AI to tailor treatments to individual patients based on their genetic makeup and medical history.

**AI in Education** is personalizing the learning experience. Adaptive learning platforms use AI to adjust the difficulty and content of lessons based on each student's individual needs. AI-powered grading systems can automate the grading of assignments, freeing up teachers' time. Virtual tutors provide personalized support and guidance to students.

**AI in Transportation** is paving the way for autonomous vehicles, which have the potential to revolutionize transportation and logistics. AI is also being used to optimize traffic flow and improve supply chain management.

Even the **Entertainment** industry is being transformed by AI. Streaming services and music platforms use AI to provide personalized recommendations to users. AI is also being used to generate music, art, and videos, opening up new possibilities for creative expression. AI-powered game characters and environments are creating more immersive and engaging gaming experiences.

## Section 4: The Future of AI

The future of **Artificial Intelligence** holds both tremendous potential and significant challenges. As AI technology continues to advance, we can anticipate even more transformative applications across various industries.

One of the primary considerations regarding the future of AI is its impact on the job market. While AI is likely to automate many existing jobs, it will also generate new opportunities in fields such as AI development, data science, and AI ethics. It is essential for individuals to acquire the skills and knowledge necessary to thrive in an AI-driven economy. The rise of AI will create many **AI jobs** but will also require workforce adaptation.

The ethical and societal implications of AI are also a major concern. As AI systems become more powerful, addressing issues such as bias, privacy, and accountability is essential. Responsible AI development and deployment are crucial to ensuring that AI benefits all of humanity.

AI also has the potential to play a significant role in addressing global challenges, such as climate change, poverty, and disease. AI can be used to develop new technologies, optimize resource allocation, and accelerate scientific discovery.

## Section 5: Getting Started with AI

If you're interested in exploring **Artificial Intelligence** further and pursuing a career in this dynamic field, numerous resources are available to help you begin.

A wide array of online courses can be found on platforms such as Coursera, edX, Udacity, and fast.ai. These courses cover diverse topics, from the fundamentals of machine learning to advanced deep learning techniques.

Additionally, many excellent books on AI and machine learning are available. Popular titles include "Hands-On Machine Learning with Scikit-Learn, Keras & TensorFlow" by Aurélien Géron and "Deep Learning" by Ian Goodfellow, Yoshua Bengio, and Aaron Courville.

To begin working with AI, you'll need to learn programming languages such as Python and R. Familiarizing yourself with AI frameworks like TensorFlow, PyTorch, and scikit-learn is also beneficial. Cloud platforms such as Google Cloud AI Platform, Amazon SageMaker, and Microsoft Azure AI offer robust tools and resources for developing and deploying AI applications.

Some popular **career paths in AI** include:

*   **Data Scientist:** Analyzes data to extract insights and build predictive models.
*   **Machine Learning Engineer:** Develops and deploys machine learning algorithms.
*   **AI Researcher:** Conducts research to advance the field of AI.
*   **AI Ethicist:** Works to ensure that AI systems are developed and used ethically and responsibly.

## Conclusion

**Artificial Intelligence** is a transformative technology that is already profoundly impacting our world. From its foundational principles to its latest trends and diverse applications, AI is reshaping industries, creating new opportunities, and challenging us to address complex ethical and societal issues.

As AI continues to evolve, it is crucial for individuals, businesses, and policymakers to understand and engage with this technology responsibly. By embracing lifelong learning, fostering ethical development practices, and promoting collaboration, we can harness the full potential of AI to create a better future for all.

Ready to dive deeper into the world of AI? Explore the resources mentioned in this article and join the conversation in the comments below! Subscribe to our newsletter for the latest AI news and insights.